# Project dashboard WIP, paged for final release


In [1]:
# Common imports from Python standard libraries
import math

# Common imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# Science imports
import scipy
from scipy import stats
from statistics import mean

# Useful imports for visualization
from plotly import subplots

# Plotly express and dash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Constants
EPS = 0.0000001
FINAL = False

# Some useful functions
def almost_equal(a,b,eps=EPS):
    return abs(a-b) <= eps

def binsearch(func, vv = 0.025, x_l = -100, x_r = 100, eps=0.00001):
    '''
    Search for x such that func(x) == vv,
    within a range of +- eps,
    with x between x_l and x_r.
    '''
    v_l = func(x_l)
    v_r = func(x_r)
    
    x_m = (x_l + x_r)/2
    v_m = func(x_m)
    
    while abs(v_m - vv) >= eps:
        if v_m < vv:
            x_l = x_m
        elif v_m > vv:
            x_r = x_m
        else:
            print("weird, this shouldn't be possible")
            return x_m
            
        v_l = func(x_l)
        v_r = func(x_r)
        x_m = (x_l + x_r)/2
        v_m = func(x_m)
    
    return x_m

# Code provided from HW4
def stdev(X):
    m = mean(X)
    return math.sqrt(sum((x-m)**2 for x in X) / len(X))

def degreesOfFreedom(X, Y):
    s1 = (stdev(X)**2)
    s2 = (stdev(Y)**2)
    df = (s1 / len(X) + s2 / len(Y))**2 / ((s1 / len(X))**2 / (len(X) - 1) + (s2 / len(Y))**2 /
(len(Y) - 1))
    return(df)

# Load data
df_full = pd.read_csv("../DATASETS/US_Accidents_Dec20_updated.csv")

df = df_full.sample(n = 10000, ignore_index = True)

# Start up app
app = dash.Dash(name="US Traffic Accidents analysis")

---
---

## (1) Boxplots, Violin Plots

In [2]:
dist_checklist_keys = ['Stop', 'Traffic_Calming', 'Roundabout', 'Traffic_Signal', 'Crossing', 'Railway', 'Bump']
distance_by_severity = {severity : np.array(df[df['Severity'] == severity]['Distance(mi)']) for severity in set(df['Severity'])}

part_1_severity = html.Div(
    [
        html.Div(
            [
                html.H2("Do traffic controls mitigate the impact of accidents?"),
                html.P('''
A serious traffic accident can leave one or more lanes of a road unusable.
After an accident, we measure the *distance* of affected road in miles.
Use this tool and explore how the severity of accidents relate to the presence of
traffic controls such as stop signs, traffic signals, roundabouts, traffic crossings,
train railways, speed-bumps, and 'traffic-calming' techniques
                '''),
                dcc.Checklist(
                    id = "severity-checklist",
                    options = [{'value' : str(x), 'label' : f'{str(x)}'}
                               for x in dist_checklist_keys],
                               #for x in [1, 2, 3, 4]],
                    value   = [],
                    labelStyle = {'display' : 'inline-block'}
                ),
            ]
        ),
        dcc.Graph(id = "severity-distance-boxplot"),
        #dcc.Link('Part 2: Histogram of weather conditions', href='/page2')
    ]
)


@app.callback(
    Output("severity-distance-boxplot", "figure"), 
    [Input("severity-checklist", "value")]
)
def generate_plot_part1(x):
    fig = px.violin(
        df,
        x = x,
        y = 'Distance(mi)',
        box = True,
        points='all'
    )
    return fig


In [3]:
weather_keys = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']

part_2_histogram = html.Div(
    [
        html.Div(
            [
                html.H2("See the distributions of different weather conditions during accidents."),
                html.P('''
Explore the distribution of different weather conditions during traffic accidents.
                '''),
            ]
        ),
        html.Div(
            [
                dcc.Graph(id = "weather-kde"),
                dcc.Checklist(
                    id = "weather-checklist",
                    options = [{'value' : str(x), 'label' : f'{str(x)}'}
                               for x in weather_keys],
                               #for x in [1, 2, 3, 4]],
                    value   = ["Temperature(F)"],
                    labelStyle = {'display' : 'inline-block'}
                ),
                html.P("Choose bin sizes:"),
                dcc.Slider(
                    id="weather-binsize",
                    min=0.1,
                    max=10.1,
                    value=1.0,
                    step = 0.1,
                    marks = {x : str(x) for x in [0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0]}
                ),
                #dcc.Link('Part 3', href='/page3')
            ]
        )
                
    ]
)


@app.callback(
    Output("weather-kde", "figure"), 
    [Input("weather-checklist", "value"),
     Input("weather-binsize", "value")
    ]
)
def generate_plot_part2(labels, binsize):
    '''
    fig = px.histogram(
        df,
        x = x,
        #marginal = 'violin',
        histnorm = 'probability density',
        nbins = bins
    )
    return fig
    '''
    data = np.nan_to_num(np.array(df[labels]), copy=True, nan=0.0, posinf=0.0, neginf=0.0).T
    fig = ff.create_distplot(data, labels, bin_size=binsize)
    return fig


In [4]:
test_keys = ['Stop', 'Crossing', 'Traffic_Calming', 'Traffic_Signal']
test_metrics = ['Distance(mi)', 'Severity']

part_3_hypothesis = html.Div(
    [
        html.Div(
            [
                html.H2("Hypothesis testing: Four different hypothesis."),
                html.P('''
Do various traffic conditions impact how bad accidents get?
Explore combinations of T-tests here and their respective p-scores.
                '''),
            ]
        ),
        html.Div(
            [
                dcc.Graph(id = "hypothesis-graph"),
                dcc.RadioItems(
                    id = "hypothesis-keys",
                    options = [{'value' : x, 'label' : f"Test how '{x}' ..."}
                               for x in test_keys],
                    value = ["Stop"],
                    labelStyle = {'display' : 'inline-block'}
                ),
                dcc.RadioItems(
                    id = "hypothesis-metric",
                    options = [{'value' : x, 'label' : f"... impacts metric '{x}'"}
                               for x in test_metrics],
                    value = ["Distance(mi)"],
                    labelStyle = {'display' : 'inline-block'}
                ),
                dcc.Link('Part 4', href='/page4')
            ]
        )
                
    ]
)

@app.callback(
    Output("hypothesis-graph", "figure"), 
    [Input("hypothesis-keys", "value"),
     Input("hypothesis-metric", "value")
    ]
)
def get_figure_part3(key = 'Stop', metric = 'Distance(mi)', show_boxes=False):
    # From our dataframe, get items with and without boolean key
    key_with = df[df[key] == True][metric]
    key_without = df[df[key] == False][metric]

    # get degrees of freedom
    v_obs = degreesOfFreedom(key_with, key_without)
    # Get the t-pdf of our two values
    pdf_with = lambda x: stats.t.pdf(
        x, df = v_obs, loc = key_with.mean(), scale = key_with.std()
    )
    pdf_without = lambda x: stats.t.pdf(
        x, df = v_obs, loc = key_without.mean(), scale = key_without.std()
    )
    # get statistics
    t_obs, p_obs = scipy.stats.ttest_ind(key_with, key_without)

    # Lines for graphing
    Xs = np.linspace(-6, 6, 1000)
    Ys_with = [pdf_with(x) for x in Xs]
    Ys_without = [pdf_without(x) for x in Xs]

    # Create figures
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=Xs, y=Ys_with, mode='lines'))
    fig.add_trace(go.Scatter(x=Xs, y=Ys_without, mode='lines'))
    fig.add_trace(
        go.Scatter(
            x = [0.0],
            y = [.50],
            text = [f"T-stat: {t_obs:.5f}    P-score: {p_obs:.5f}"],
            mode = "text"
        )
    )
    
    if show_boxes:
        fig.add_trace(go.Box(x=key_with))
        fig.add_trace(go.Box(x=key_without))
    
    return fig


In [5]:
part_4_map = html.Div(
    [
        html.Div(
            [
                html.H2("Map"),
                html.P('''
The dataset labels accidents by the severity of the imapct it has on traffic,
on a scale of 1 (not severe) to 4 (most severe). Explore the distribution of
severity of these accidents here, and how they relate with different traffic measures.
                '''),
            ]
        ),
        html.Div(
            [
                dcc.Graph(id = "accidents-map"),
                dcc.RadioItems(
                    id = "map-severity",
                    options = [{'value' : x, 'label' : f"Severity {x}"}
                               for x in [1,2,3,4]],
                    value = 1,
                    labelStyle = {'display' : 'inline-block'}
                ),
                dcc.RadioItems(
                    id = "severity-checklist-map",
                    options = [{'value' : str(x), 'label' : f'{str(x)}'}
                               for x in ['Any'] + dist_checklist_keys],
                               #for x in [1, 2, 3, 4]],
                    value   = 'Any',
                    labelStyle = {'display' : 'inline-block'}
                ),
                dcc.Link('Part 5', href='/page1')
            ]
        )
                
    ]
)

@app.callback(
    Output("accidents-map", "figure"), 
    [Input("map-severity", "value"),
     Input("severity-checklist-map", "value"),
    ]
)
def get_figure_part4(severity = [1], traffic_measure = 'Any'):
    colors = [
        'rgba(0,0,0,0)',
        'rgba(0,128,192,192)',
        'rgba(128,0,192,192)',
        'rgba(256,0,128,192)', 
        'rgba(256,128,0,192)'
    ]
    
    if traffic_measure == 'Any':
        df_subset = df[df['Severity'] == severity]
    else:
        df_subset = df[df['Severity'] == severity][df[traffic_measure] == True]
    
    color_str = colors[severity]
    color = [color_str for _ in range(len(df_subset))]
    
    fig = go.Figure(
        data = go.Scattergeo(
            lat=df_subset['Start_Lat'],
            lon=df_subset['Start_Lng'],
            marker_color = color,
            marker_size = 4,
        )
    )
    
    fig.update_layout(geo_scope='usa')
    
    return fig


In [9]:
part_video = html.Video(
    controls = True,
    id = 'movie_player',
    src = "./static/demo_video.mp4",
    autoPlay=True
)

app.layout = html.Div(
    [
        part_video,
        html.Br(),
        part_1_severity,
        html.Br(),
        part_2_histogram,
        html.Br(),
        part_3_hypothesis,
        html.Br(),
        part_4_map,
        html.Br(),
    ]
)

app.run_server(port = 10001, debug=False)

Dash is running on http://127.0.0.1:10001/

 * Serving Flask app 'US Traffic Accidents analysis' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:10001/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Dec/2021 23:38:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:38:27] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:38:27] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:38:27] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:38:27] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:38:27] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:38:27] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:38:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:38:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:38:28] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/dash/_callback.py", line 151, 

127.0.0.1 - - [10/Dec/2021 23:38:28] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [10/Dec/2021 23:38:28] "GET /static/demo_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [10/Dec/2021 23:38:30] "GET /static/demo_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [10/Dec/2021 23:38:30] "GET /static/demo_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [10/Dec/2021 23:38:30] "GET /static/demo_video.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [10/Dec/2021 23:38:32] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:38:32] "GET /static/demo_video.mp4 HTTP/1.1" 206 -


## Running them all in pages (WIP)

In [27]:
part_1_severity
part_2_histogram
part_3_hypothesis
part_4_map

Div([Div([H2('Map'), P('\nThe dataset labels accidents by the severity of the imapct it has on traffic,\non a scale of 1 (not severe) to 4 (most severe). Explore the distribution of\nseverity of these accidents here, and how they relate with different traffic measures.\n                ')]), Div([Graph(id='accidents-map'), RadioItems(id='map-severity', labelStyle={'display': 'inline-block'}, options=[{'value': 1, 'label': 'Severity 1'}, {'value': 2, 'label': 'Severity 2'}, {'value': 3, 'label': 'Severity 3'}, {'value': 4, 'label': 'Severity 4'}], value=1), RadioItems(id='severity-checklist-map', labelStyle={'display': 'inline-block'}, options=[{'value': 'Any', 'label': 'Any'}, {'value': 'Stop', 'label': 'Stop'}, {'value': 'Traffic_Calming', 'label': 'Traffic_Calming'}, {'value': 'Roundabout', 'label': 'Roundabout'}, {'value': 'Traffic_Signal', 'label': 'Traffic_Signal'}, {'value': 'Crossing', 'label': 'Crossing'}, {'value': 'Railway', 'label': 'Railway'}, {'value': 'Bump', 'label': 'Bu

In [48]:
page_links = html.Li(
    [
        dcc.Link(' Part 1: Traffic control measures and severity   ',
                 href='/page1'),
        dcc.Link(' Part 2: Histogram of weather conditions   ',
                 href='/page2'),
        dcc.Link(' Part 3: A collection of hypothesis tests   ',
                 href='/page3'),
        dcc.Link(' Part 4: Map exploration   ',
                 href='/page4'),
    ]
)

app_pages = {
    '/'      : app_index,
    '/page1' : part_1_severity,
    '/page2' : part_2_histogram,
    '/page3' : part_3_hypothesis,
    '/page4' : part_4_map
}

@app.callback(Output('page-content', 'children'),[Input('url', 'pathname')])
def display_page(pathname):
    print(f"!!! {pathname}")
    return app_pages[pathname]


app.layout = html.Div(
    [
        dcc.Location(id='url', refresh=False),
        html.Div(id='page-content'),
        links_to_pages
    ]
)

app.run_server(port = 10001, debug=False)

Dash is running on http://127.0.0.1:10001/

Dash is running on http://127.0.0.1:10001/

 * Serving Flask app 'US Traffic Accidents analysis' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:10001/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Dec/2021 23:10:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:10:46] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:10:46] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:10:46] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 23:10:47] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -


# Okay. Let's try something else.